# This script is brought to by ROAST Labs the internal development and innovation team at [ROAST](https://weareroast.com/) a digital marketing agency.

**Purpose:**

This Python script interfaces with the AccuRanker API to fetch domain information and keywords associated with a selected domain. It checks the HTTP status of URLs linked with each keyword and logs this data into a CSV file along with other keyword details.

**How to use**

1.   Add in your [Accuranker API key on line 5](https://app.accuranker.com/app/api?_gl=1*2h0ngs*_ga*MjAxODIxNjAzOS4xNzE1MDkzNjgx*_ga_D2SXNT78V1*MTcxNTA5MzY4MC4xLjEuMTcxNTA5MzY4Mi41OC4wLjE5NjI4MzIyMjE.#section/Introduction).
2.   Run the code
3.   Select a domain by type a number from the domains listed
4.   Script will say how many key phrases are be checked, type yes to continue
5.   The script will run and then save a csv in the sample_data folder.


PLease note you run this script at your own risk, ROAST has no liability for the running of this or any other scripts.

In [ ]:
import requests
import csv
from datetime import datetime

# Constants
ACCURANKER_API_TOKEN = 'insert-you-api-key-here'
HEADERS = {
    'Authorization': f'Token {ACCURANKER_API_TOKEN}',
    'Content-Type': 'application/json'
}

def get_domains():
    url = 'https://app.accuranker.com/api/v4/domains/?fields=domain,display_name,id'
    response = requests.get(url, headers=HEADERS)
    response.raise_for_status()
    return response.json()

def select_domain(domains):
    for idx, domain in enumerate(domains, 1):
        print(f"{idx}. {domain['display_name']} ({domain['domain']})")
    selection = int(input("Select a domain by number: "))
    return domains[selection - 1]['id']

def get_keywords(domain_id):
    url = f'https://app.accuranker.com/api/v4/domains/{domain_id}/keywords/?fields=id,keyword,search_type,ranks.id,ranks.rank,ranks.page_serp_features.ads_top,ranks.page_serp_features.ads_bottom,ranks.highest_ranking_page,ranks.created_at,ranks.rank,ranks.extra_ranks'
    response = requests.get(url, headers=HEADERS)
    response.raise_for_status()
    return response.json()  # This returns a list directly

def check_http_status(url):
    if url is None:
        return "no check"
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.36'
    }
    try:
        response = requests.get(url, headers=headers, timeout=10)  # GET request with timeout
        return response.status_code
    except requests.RequestException as e:
        return str(e)

def process_keyword_data(keywords):
    print(f"Total keywords to be checked: {len(keywords)}")
    confirm = input("Do you want to proceed with URL checks? (yes/no): ")
    if confirm.lower() != 'yes':
        return []

    processed_data = []
    status_counts = {}
    total_checked = 0

    for i, keyword in enumerate(keywords, 1):
        for rank in keyword['ranks']:
            total_checked += 1
            highest_ranking_page_status = check_http_status(rank['highest_ranking_page'])
            status_counts[highest_ranking_page_status] = status_counts.get(highest_ranking_page_status, 0) + 1
            row_data = {
                'id': keyword['id'],
                'keyword': keyword['keyword'],
                'search_type': keyword['search_type'],
                'rank_id': rank['id'],
                'created_at': rank['created_at'],
                'rank': rank['rank'],
                'highest_ranking_page': rank['highest_ranking_page'],
                'highest_ranking_page_status': highest_ranking_page_status,
                'ads_top': rank['page_serp_features']['ads_top'],
                'ads_bottom': rank['page_serp_features']['ads_bottom'],
            }
            processed_data.append(row_data)
            print(f"Checked {total_checked}/{len(keywords)} keywords...")
    print("URL check complete.")
    print(f"Total URLs checked: {total_checked}")
    for status, count in status_counts.items():
        print(f"HTTP {status}: {count} times")

    return processed_data

def write_to_csv(data):
    date_time = datetime.now().strftime("%y%m%d %H %M")
    filename = f"/content/sample_data/Accuranker Ranking Page Status Check {date_time}.csv"

    with open(filename, mode='w', newline='') as file:
        writer = csv.writer(file)

        # Write the header
        header = [
            'id', 'keyword', 'search_type', 'rank_id', 'created_at', 'rank',
            'highest_ranking_page', 'highest_ranking_page_status', 'ads_top', 'ads_bottom'
        ]
        writer.writerow(header)

        # Write the rows
        for keyword_data in data:
            row = [
                keyword_data['id'], keyword_data['keyword'], keyword_data['search_type'],
                keyword_data['rank_id'], keyword_data['created_at'], keyword_data['rank'],
                keyword_data['highest_ranking_page'], keyword_data['highest_ranking_page_status'],
                keyword_data['ads_top'], keyword_data['ads_bottom']
            ]
            writer.writerow(row)
    print(f"Data saved to {filename}")

# Main execution
domains = get_domains()
domain_id = select_domain(domains)
keywords = get_keywords(domain_id)
processed_data = process_keyword_data(keywords)
write_to_csv(processed_data)
